In [ ]:
%cd "/content/drive/MyDrive/cow"

/content/drive/MyDrive/cow


In [ ]:
!dir

drive  sample_data


In [ ]:
#Unzip contents to folder
#!unzip "/content/drive/MyDrive/cow/test.zip" -d "/content/drive/MyDrive/cow"

In [ ]:
#Unzip contents to folder
#!unzip "/content/drive/MyDrive/cow/train.zip" -d "/content/drive/MyDrive/cow/train"

In [ ]:
#Unzip contents to folder
#!unzip "/content/drive/MyDrive/cow/val.zip" -d "/content/drive/MyDrive/cow/val"

In [1]:
import argparse

import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler

import matplotlib.pyplot as plt

from torchvision import transforms, datasets

In [2]:
data_dir = "/content/drive/MyDrive/cow"
ckpt_dir = "/content/drive/MyDrive/cow/checkpoint_unet_less_param"
log_dir = "/content/drive/MyDrive/cow/log"
early_ckpt_dir = "/content/drive/MyDrive/cow/earlystop_unet_less_param"

In [3]:
class Dataset(Dataset):
  def __init__(self, data_dir, transform = None):
    self.data_dir = data_dir
    self.transform = transform

    lst_data = os.listdir(self.data_dir)
    lst_label = [f for f in lst_data if f.startswith('label')]
    lst_input = [f for f in lst_data if f.startswith('input')]

    lst_label.sort()
    lst_input.sort()

    self.lst_label = lst_label
    self.lst_input = lst_input

  def __len__(self):
    return len(self.lst_label)

  def __getitem__(self, index):
    label = np.load(os.path.join(self.data_dir, self.lst_label[index]))
    input = np.load(os.path.join(self.data_dir, self.lst_input[index]))

    #normalize
    label = label/255.0
    input = input/255.0

    if label.ndim == 2:
      label = label[:, :, np.newaxis]
      
    if input.ndim == 2:
      input = input[:, :, np.newaxis]

    data = {'input' : input, 'label': label}
  
    if self.transform:
      data = self.transform(data)
    
    return data


In [4]:
class ToTensor(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        label = label.transpose((2, 0, 1)).astype(np.float32)
        input = input.transpose((2, 0, 1)).astype(np.float32)

        data = {'label': torch.from_numpy(label), 'input': torch.from_numpy(input)}

        return data
class Normalization(object):
  def __init__(self,mean=0.5, std = 0.5):
    self.mean = mean
    self.std = std

  def __call__(self, data):
    label, input = data['label'], data['input']
    input = (input - self.mean) / self.std
    data = {'label' : label, 'input' : input}
    return data

class RandomFlip(object):
  def __call__(self,data):
    label, input = data['label'], data['input']

    if np.random.rand() > 0.5:
      input = np.fliplr(input)
      label = np.fliplr(label)

    if np.random.rand() > 0.5:
      input = np.flipud(input)
      label = np.flipud(label)
    
    data = {'label' : label, 'input' : input} 

    return data


In [5]:
import os
import numpy as np
import torch
import torch.nn as nn


## 네트워크 구축
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)

            return cbr

        # Contracting path
        self.enc1_1 = CBR2d(in_channels=3, out_channels=64)
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=128, out_channels=128)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)

        self.pool4 = nn.MaxPool2d(kernel_size=2)

        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)

        # Expansive path
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)

        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512)
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)

        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)

        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)

        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)

        self.fc = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)

    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)
        dec5_1 = self.dec5_1(enc5_1)

        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)

        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_2), dim=1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        x = self.fc(dec1_1)

        return x

In [6]:
#Parameters

lr = 1e-3
batch_size = 4
num_epoch = 100

device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Model_name = "cow_segmentation_unet_less_param"

In [7]:
#New Cell

def dice_coef_metric(pred, label):
    intersection = 2.0 * (pred * label).sum()
    union = pred.sum() + label.sum()
    if pred.sum() == 0 and label.sum() == 0:
        return 1.
    return intersection / union
def dice_coef_loss(pred, label):
    smooth = 1.0
    intersection = 2.0 * (pred * label).sum() + smooth
    union = pred.sum() + label.sum() + smooth
    return 1 - (intersection / union)
def bce_dice_loss(pred, label):
    dice_loss = dice_coef_loss(pred, label)
    bce_loss = nn.BCELoss()(pred, label)
    return dice_loss + bce_loss


## Intersection Over Union (IOU) 구현
def IOU_Numpy(outputs: torch.Tensor, labels: torch.Tensor, reduction='mean', SMOOTH=1e-6):
    # You can comment out this line if you are passing tensors of equal shape
    # But if you are passing output from UNet or something it will most probably
    # be with the BATCH x 1 x H x W shape
    #outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
    results = []

    outputs = outputs.squeeze(1)
    labels = labels.squeeze(1)
    
    outputs = outputs.to('cpu').detach().numpy()
    labels = labels.to('cpu').detach().numpy()
    
    batch_size = labels.shape[0]
    for batch in range(batch_size):
        t, p = labels[batch], outputs[batch]
        true = np.sum(t)
        pred = np.sum(p)
        
        # non empty mask case.  Union is never empty 
        # hence it is safe to divide by its number of pixels
        intersection = np.sum(t * p)
        union = true + pred - intersection
        #iou = (intersection + SMOOTH) / (union + SMOOTH)
        iou = intersection / union        
        results.append(iou)

    if reduction == 'mean':
        return np.mean(results)  # Or thresholded.mean() if you are interested in average across the batch
    else:
        return results   

In [8]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, counter=0, best_score = None, val_loss_min = np.inf, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = counter
        self.best_score = best_score
        self.early_stop = False
        self.val_loss_min = val_loss_min
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, models_dict):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, models_dict)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, models_dict)
            self.counter = 0

    def save_checkpoint(self, val_loss, models_dict):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        for file_name in models_dict.keys():
            torch.save(models_dict[file_name], self.path + "/" + file_name + ".pkl")
        self.val_loss_min = val_loss


In [9]:
#Train net
transform = transforms.Compose([Normalization(mean = 0.5, std = 0.5), RandomFlip(), ToTensor()])

dataset_train = Dataset(data_dir=os.path.join(data_dir, 'train'),transform = transform)
loader_train = DataLoader(dataset_train, batch_size = batch_size, shuffle =True, num_workers = 2)
dataset_val = Dataset(data_dir = os.path.join(data_dir, 'val'), transform = transform)
loader_val = DataLoader(dataset_val, batch_size = batch_size, shuffle = False, num_workers =2)

net = UNet().to(device)
fn_loss = nn.BCEWithLogitsLoss().to(device)
optim = torch.optim.Adam(net.parameters(), lr= lr)

#New Line
optimizer_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer = optim, mode = "min", factor = 0.5, patience = 5, verbose = True)

num_data_train = len(dataset_train)
num_data_val = len(dataset_val)

num_batch_train = np.ceil(num_data_train / batch_size)
num_batch_val = np.ceil(num_data_val / batch_size)

##function 설정
fn_tonumpy = lambda x:x.to('cpu').detach().numpy().transpose(0,2,3,1)
fn_denorm = lambda x, mean, std : (x * std) + mean
fn_class = lambda x: 1.0 * (x>0.5)

##TensorBoard 사용
writer_train = SummaryWriter(log_dir = os.path.join(log_dir, 'train'))
writer_val = SummaryWriter(log_dir = os.path.join(log_dir, 'val'))


##네트워크 저장
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    torch.save({'net': net.state_dict(), 'optim': optim.state_dict(), 'es_counter': early_stopping.counter, 'v_loss_min' : early_stopping.val_loss_min, 'best_score' : early_stopping.best_score},"%s/model_epoch%d.pth" % (ckpt_dir, epoch))

##네트워크 호출
def load(ckpt_dir, net, optim):
    if not os.path.exists(ckpt_dir):
        epoch = 0
        return net, optim, epoch, 0, None, np.inf

    ckpt_lst = os.listdir(ckpt_dir)
    print(ckpt_lst)
    ckpt_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    dict_model = torch.load('%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])
    es_counter = dict_model['es_counter']
    best_score = dict_model['best_score']
    v_loss_min = dict_model['v_loss_min']

    return net, optim, epoch, es_counter, best_score, v_loss_min 

#네트워크 학습
st_epoch = 0
st_es_counter = 0
st_best_score = None
st_val_min_loss = np.inf
#학습된게 있다면 거기서 부터 진행함
net, optim, st_epoch, st_es_counter, st_best_score, st_val_min_loss = load(ckpt_dir = ckpt_dir, net = net, optim = optim)
early_stopping = EarlyStopping(patience=15, verbose=True, counter = st_es_counter, best_score = st_best_score, val_loss_min = st_val_min_loss, path=early_ckpt_dir)
print("Train starts from : Epoch %04d / %04d | EarlyStopping Counter %04d / %04d"% (st_epoch, num_epoch, st_es_counter, early_stopping.patience))


for epoch in range(st_epoch + 1, num_epoch+1):
    net.train()  #net가 train이라는 것을 알려줌
    loss_arr = []
    train_iou_arr = []
    for batch, data in enumerate(loader_train, 1):
        #forward pass
        label = data['label'].to(device)
        input = data['input'].to(device)
        output = net(input)
        
        print(output.shape)
        print(label.shape)

        #backward pass
        optim.zero_grad()
        loss = fn_loss(output, label)
        loss.backward()
        optim.step()

        #loss function cal
        loss_arr += [loss.item()]
        train_iou_arr += [IOU_Numpy(fn_class(output), label)]
        print("Train : Epoch %04d / %04d | Batch %04d / %04d | LOSS %.4f | IOU_MEAN %.4f"%
        (epoch, num_epoch, batch, num_batch_train, np.mean(loss_arr), np.mean(train_iou_arr)))

        #TensorBoard 저장
        label = fn_tonumpy(label)
        input = fn_tonumpy(fn_denorm(input, mean = 0.5, std = 0.5))
        output = fn_tonumpy(fn_class(output))

        writer_train.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats = 'NHWC')
        writer_train.add_image('input', input, num_batch_train * (epoch - 1) + batch, dataformats = 'NHWC')
        writer_train.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats = 'NHWC')
    writer_train.add_scalar('loss', np.mean(loss_arr), epoch)


    with torch.no_grad():
        net.eval() #validate 과정 명시
        loss_arr = []
        #New line
        val_iou_arr = []
        for batch, data in enumerate(loader_val, 1):
            #forward pass
            label = data['label'].to(device)
            input = data['input'].to(device)
            output = net(input)
            #val은 backward 필요없음
            #손실함수
            loss = fn_loss(output, label)
            loss_arr += [loss.item()]
            val_iou_arr += [IOU_Numpy(fn_class(output), label)]
            #New line
            print("Valid : Epoch %04d / %04d | Batch %04d / %04d | LOSS %.4f | IOU_MEAN %.4f"%
            (epoch, num_epoch, batch, num_batch_val, np.mean(loss_arr), np.mean(val_iou_arr)))

            #Tensorboard 저장
            label = fn_tonumpy(label)
            input = fn_tonumpy(fn_denorm(input, mean = 0.5, std = 0.5))
            output = fn_tonumpy(fn_class(output))
                
            writer_val.add_image('label', label, num_batch_val * (epoch - 1) + batch, dataformats = 'NHWC')
            writer_val.add_image('input', input, num_batch_val * (epoch - 1) + batch, dataformats = 'NHWC')
            writer_val.add_image('output', output, num_batch_val * (epoch - 1) + batch, dataformats = 'NHWC')

    #New Line
    #Optimizer update
    optimizer_scheduler.step(1-np.mean(val_iou_arr))
    writer_val.add_scalar('loss', np.mean(loss_arr), epoch)
    
    save(ckpt_dir = ckpt_dir, net= net, optim = optim, epoch = epoch)

    writer_train.close()
    writer_val.close()

    #Early Stopping
    models_dict = dict()
    models_dict[Model_name] = net.state_dict()
    early_stopping(1 - np.mean(val_iou_arr), models_dict)
    
    state_dict = torch.load(early_ckpt_dir + '/' + Model_name + ".pkl", map_location=device)
    # Early stopping patient가 초과되면 멈춤
    if early_stopping.early_stop:
        break

print("done!")

        

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
torch.Size([4, 1, 512, 512])
torch.Size([4, 1, 512, 512])
Train : Epoch 0043 / 0100 | Batch 2687 / 4058 | LOSS 0.0500 | IOU_MEAN 0.8770
torch.Size([4, 1, 512, 512])
torch.Size([4, 1, 512, 512])
Train : Epoch 0043 / 0100 | Batch 2688 / 4058 | LOSS 0.0500 | IOU_MEAN 0.8770
torch.Size([4, 1, 512, 512])
torch.Size([4, 1, 512, 512])
Train : Epoch 0043 / 0100 | Batch 2689 / 4058 | LOSS 0.0500 | IOU_MEAN 0.8770
torch.Size([4, 1, 512, 512])
torch.Size([4, 1, 512, 512])
Train : Epoch 0043 / 0100 | Batch 2690 / 4058 | LOSS 0.0500 | IOU_MEAN 0.8770
torch.Size([4, 1, 512, 512])
torch.Size([4, 1, 512, 512])
Train : Epoch 0043 / 0100 | Batch 2691 / 4058 | LOSS 0.0500 | IOU_MEAN 0.8770
torch.Size([4, 1, 512, 512])
torch.Size([4, 1, 512, 512])
Train : Epoch 0043 / 0100 | Batch 2692 / 4058 | LOSS 0.0500 | IOU_MEAN 0.8770
torch.Size([4, 1, 512, 512])
torch.Size([4, 1, 512, 512])
Train : Epoch 0043 / 0100 | Batch 2693 / 4058 | LOSS 0.0500 | IOU_MEAN 0.8770
torch.Size([